MBTdelay -- determine stations names and get weather data from Dark Sky API

© Mark Mace 2019 markfmace@gmail.com


In [1]:
# GENERAL INCLUSIONS
import numpy as np
import requests
import os
import json
import time
import datetime
import matplotlib.pyplot as plt
import csv
import pandas as pd

# FOR DATES AND TIMES #
import time
import datetime
from dateutil import tz
from datetime import timedelta
import arrow

In [2]:
# IMPORT API KEYS -- OTHERWISE NEED TO BE SET IN NOTEBOOK
from MY_API_KEYS import *


In [12]:
# GENERAL PURPOSE FUNCTIONS FOR CONVERSION TO SECONDS
def weeks(x):
    return int(7*24*60*60*x)

def days(x):
    return int(24*60*60*x)

def hours(x):
    return int(60*60*x)

def minutes(x):
    return int(60*x)

# CREAT FORMATTING STRING OF PYTHON JSON OBJECT
def jprint(obj):
    text = json.dumps(obj, sort_keys=True)
    print(text)

# HH:MM TO s
def time_to_unix(input_text_time):
    hr_loc,min_loc=str(input_text_time).split(":")
    return int(hr_loc)*60*60+int(min_loc)*60

# FOR HOUR BINNING OF ENTRY DATA
def rounddown(x):
    return int(np.floor(int(x)/100.0))*100

## CONVERT HUMAN READABLE TO UNIX TIMESTAMP   
# CONVERT YYYY-mm-dd HH:MM:SS (EASTERN) TO UNIX (utc) TIMESTAMP
def conv_east_to_unixts_hms(dt):
    east=arrow.get(dt,'YYYY-MM-DD HH:mm:ss').replace(tzinfo='US/Eastern')     # NOW HAS TZ ATTACHED
    # RETURN UNIX TIMESTAMP
    return east.timestamp

# CONVERT YYYY-mm-dd HH:MM:SS (UTC/GMT) TO UNIX (utc) TIMESTAMP
def conv_utc_to_unixts_hms(dt):
    utc=arrow.get(dt,'YYYY-MM-DD HH:mm:ss').replace(tzinfo='UTC')     # NOW HAS TZ ATTACHED
    # RETURN UNIX TIMESTAMP
    return utc.timestamp
    
# CONVERT YYYY-mm-dd HH:MM (EASTERN) TO UNIX (utc) TIMESTAMP
def conv_east_to_unixts_hm(dt):
    east=arrow.get(dt,'YYYY-MM-DD HH:mm').replace(tzinfo='US/Eastern')     # NOW HAS TZ ATTACHED
    # RETURN UNIX TIMESTAMP
    return east.timestamp

# CONVERT YYYY-mm-dd (EASTERN) TO UNIX (utc) TIMESTAMP
def conv_east_to_unixts_dateonly(dt):
    east=arrow.get(dt,'YYYY-MM-DD').replace(tzinfo='US/Eastern')     # NOW HAS TZ ATTACHED
    # RETURN UNIX TIMESTAMP
    return east.timestamp

# CONVERT YYYY-mm-dd HH:MM (UTC/GMT) TO UNIX (utc) TIMESTAMP
def conv_utc_to_unixts_hm(dt):
    utc=arrow.get(dt,'YYYY-MM-DD HH:mm').replace(tzinfo='UTC')     # NOW HAS TZ ATTACHED
    # RETURN UNIX TIMESTAMP
    return utc.timestamp

# CONVERT HUMAN READABLE BETWEEN TIMEZONES    
# CONVERT YYYY-mm-dd HH:MM:SS (EASTERN) TO YYYY-mm-dd HH:MM:SS (UTC)
def conv_east_to_utc_hms(dt):
    east=arrow.get(dt,'YYYY-MM-DD HH:mm:ss').replace(tzinfo='US/Eastern')     # NOW HAS TZ ATTACHED
    utc=east.to('UTC')
    return utc.format('YYYY-MM-DD HH:mm:ss')

# CONVERT YYYY-mm-dd HH:MM:SS (UTC) TO YYYY-mm-dd HH:MM (EASTERN)
def conv_utc_to_east_hms(dt):
    utc=arrow.get(dt,'YYYY-MM-DD HH:mm:ss').replace(tzinfo='UTC')     # NOW HAS TZ ATTACHED
    east=utc.to('US/Eastern')
    return east.format('YYYY-MM-DD HH:mm:ss')
    
# CONVERT YYYY-mm-dd HH:MM (EASTERN) TO YYYY-mm-dd HH:MM (UTC)
def conv_east_to_utc_hm(dt):
    east=arrow.get(dt,'YYYY-MM-DD HH:mm').replace(tzinfo='US/Eastern')     # NOW HAS TZ ATTACHED
    utc=east.to('UTC')
    return utc.format('YYYY-MM-DD HH:mm')

# CONVERT YYYY-mm-dd HH:MM (UTC) TO YYYY-mm-dd HH:MM (EASTERN)
def conv_utc_to_east_hm(dt):
    utc=arrow.get(dt,'YYYY-MM-DD HH:mm').replace(tzinfo='UTC')     # NOW HAS TZ ATTACHED
    east=utc.to('US/Eastern')
    return east.format('YYYY-MM-DD HH:mm')

## CONVERT UNIX TIMESTAMP TO HUMAN READABLE     
# CONVERT UNIX (utc) TIMESTAMP TO YYYY-mm-dd HH:MM:SS (UTC)
def conv_unixts_to_utc_hms(ts):
    utc=arrow.Arrow.fromtimestamp(ts).to('UTC')
    return utc.format('YYYY-MM-DD HH:mm:ss')

# CONVERT UNIX (utc) TIMESTAMP TO YYYY-mm-dd HH:MM:SS (EASTERN)
def conv_unixts_to_east_hms(ts):
    east=arrow.Arrow.fromtimestamp(ts).to('US/Eastern')
    return east.format('YYYY-MM-DD HH:mm:ss')

# CONVERT UNIX (utc) TIMESTAMP TO YYYY-mm-dd HH:MM (UTC)
def conv_unixts_to_utc_hm(ts):
    utc=arrow.Arrow.fromtimestamp(ts).to('UTC')
    return utc.format('YYYY-MM-DD HH:mm')

# CONVERT UNIX (utc) TIMESTAMP TO YYYY-mm-dd HH:MM (EASTERN)
def conv_unixts_to_east_hm(ts):
    east=arrow.Arrow.fromtimestamp(ts).to('US/Eastern')
    return east.format('YYYY-MM-DD HH:mm')

# RETURNS DAY OF WEEK 
# M-0, Tu-1, W-2, Th-3 F-4 Sa-6 Su-7
# TAKES YYYY-MM-DD HH:MM:SS RETURNS Day (IN WHATEVER TIMEZONE)
def get_day_of_week(dt):
    dtt = arrow.get(dt)
    return dtt.weekday()

# TAKES UNIX TS RETURNS Day (IN EASTERN/BOSTON)
def get_day_of_week_east_unix(ts):
    east=conv_unixts_to_east_hms(ts)
    return get_day_of_week(east)
    
# TAKES UNIX TS RETURNS Day (IN UTC)
def get_day_of_week_utc_unix(ts):
    utc=conv_unixts_to_utc_hms(ts)
    return get_day_of_week(utc)
    


In [13]:
def matcher(x):
    for i in green_line_d:
        if i.lower() in x.lower():
            return True
        else:
            return False

In [14]:

green_line_d=["Lechmere", "Science Park", "North Station",
              "Haymarket", "Government Center", "Park Street", "Boylston",
             "Arlington", "Copley","Hynes", "Kenmore","Fenway","Longwood",
             "Brookline Village", "Brookline Hills", "Beaconsfield", "Reservoir",
             "Chestnut Hill", "Newton Centre", "Newton Highlands", "Eliot",
             "Waban","Woodland", "Riverside"]


In [15]:
####################################
# LIST OF ALL STOPS PROVIDED BY MBTA 
####################################
gld=pd.read_csv('DATA/GreenLineDStops.csv')


# NEEDED FOR FILTERING -- THIS IS PREFILTERED #
# FIRST FILL OUT THE GREEN LINE BY UPTOWN AND DOWNTOWN 
# HARDCODED TO BE CORRECT
# NOTE THAT LAST NAME IS DESCRIPTION IS DESTINATION
# stops_list=pd.read_csv('DATA/stops.csv')
# REMOVE UNNCESSARY COLUMNS AND GET ONLY MBTA RAIL RAPID TRANSIT
# stops_list=stops_list[["stop_id","stop_code","stop_name","stop_desc","stop_lat","stop_lon","location_type"]]
# stops_list=stops_list[np.logical_and(stops_list['location_type']==0.0,stops_list['stop_desc'].str.contains("Green Line",na=False))]
# stops_list=stops_list.drop('location_type',axis=1)
# stops_list.to_csv("DATA/GreenLineStops.csv")
# gld=gld[gld['stop_name'].str.contains('|'.join(green_line_d))]

# MAKES NAMES FRIENDLIER FOR USE
# Quincy Center->QuincyCenter
gld['stop_name']=gld['stop_name'].str.replace(" ","")

# NORTHBOUND -- ODD NUMBERS
# GREEN LINE LECHMERE BOUND FROM RIVERSIDE
gl_lm_frs=gld[np.logical_or(gld['direction']==1,gld['direction']==-1)]

# SOUTHBOUND
# GRENE LINE RIVERSIDE BOUND FROM LECHMERE
gl_rs_flm=gld[np.logical_or(gld['direction']==0,gld['direction']==-1)]
# gl_rs_flm=gl_rs_flm.sort_values(by=['stop_id'])

# SHOULD BE  -- ALL CHECKED #
print(len(gl_lm_frs),len(gl_rs_flm))

# # LIST ALL STATIONS ONCE -- UNIQUE FOR NAMES 
all_stations=gl_lm_frs
all_stations_name=np.unique(all_stations['stop_name'])

# LIST ALL STATIONS ONCE -- UNIQUE FOR NAMES 
all_stations_complete=[gl_lm_frs,gl_rs_flm]
all_stations_complete=pd.concat(all_stations_complete)
all_stations_complete_id=np.unique(all_stations_complete['stop_id'])



25 28


In [43]:
# gl_rs_flm.values

In [21]:
# GET AND SAVE WEATHER DATA FROM DARK SKY FOR A GIVEN POSITION AND DATE RANGE
# DATES ARE DEFINED FOR THE LOCAL TIME 
def GetWeatherData(start_date,end_date,lat,long):

    base_url='https://api.darksky.net/forecast/'+MY_DS_KEY+'/'+str(lat)+','+str(long)
    # DEFINE START AND END TIME
    start=conv_east_to_unixts_dateonly(start_date)
    end  =conv_east_to_unixts_dateonly(end_date)
    #print(start,end)
    #print(end-start)

    # DETERMINE NUMBER OF DAYS TO GET WEATHER #
    num_days=int((end-start)/(24*60*60))
    print("# NUMBER OF DAYS =",num_days)
    
    all_days=[] # STORAGE OF WEATHER DATA FOR ALL DAYS REQUESTED 
    
    # PULL WEATHER DATA FOR ALL DAYS #
    for i in range(0,num_days):

        loc_date=str(start+days(i))

        request_URL=base_url+','+loc_date
    
        #tcheck=datetime.datetime.utcfromtimestamp(int(iDate)).strftime('%Y-%m-%d %H:%M:%S')
        # API REQUEST
        response=requests.get(request_URL)
        day_data=response.json()["hourly"]["data"]
        
        t=0
        temp=0.0
        precip_int=0.0
        precip_pro=0.0
        precip_acc=0.0
        precip_typ='None'
        
        # LOOP THROUGH ALL HOURS IN THE DAY #
        # STORE TIME, TEMPTAUTRE, PRECIPIATION INTENSITY AND PROBA
        for hour_data in day_data:
            
            # THIS IS FOR THE VERY RARE CIRCUMSTANCE THAT THERE IS NO DS DATA
            t=hour_data['time'] # UTC TIME #
            try:
                temp=hour_data['temperature']
            except BaseException:  
                print("NO temperature GIVEN!!")
                dropped_data_flag=1
                continue

            try:
                precip_int=hour_data['precipIntensity']
            except BaseException:
                print("NO precipIntensity GIVEN!!")
                continue
            try:
                precip_pro=hour_data['precipProbability']
            except BaseException:
                print("NO precipProbability GIVEN!!")
                continue
                
            ## THESE ARE VARIARBLES WHICH DS DOESNT POPULATE IF NONZERO   
            try:
                precip_acc=hour_data['precipAccumulation']
            except BaseException:
                precip_acc=0.0
                
            try:
                precip_typ=hour_data['precipType']
            except BaseException:
                precip_typ='None'
                        
            all_days.append([t,temp,precip_int,precip_pro,precip_acc,precip_typ])
            
    # RETURN ARRAY OF ALL WEATHER DATA FOR DAYS AND LOCATION 
    return all_days


In [46]:
print(all_stations_name[1:])

['Beaconsfield' 'Boylston' 'BrooklineHills' 'BrooklineVillage'
 'ChestnutHill' 'Copley' 'Eliot' 'Fenway' 'GovernmentCenter' 'Haymarket'
 'HynesConventionCenter' 'Kenmore' 'Lechmere' 'Longwood' 'NewtonCentre'
 'NewtonHighlands' 'NorthStation' 'ParkStreet' 'Reservoir' 'Riverside'
 'SciencePark' 'Waban' 'Woodland']


In [47]:
# GET WEATHER DATA FOR STATIONS #
# THIS IS AN INTENSE SCRIPT AND SHOULD ONLY BE RUN TO COLLECT FINAL DATA #
date_init="2016-01-01"
date_fina="2019-09-01"
# SOMETHING WENT WRONG WHEN I ASKED FOR 2018-06-31
# ASSEMBLE ALL SATIONS

# https://api.darksky.net/forecast/c1f27e675bba344948f6b45dcdd6e7e6/42.395428,-71.142483,1528369200

# PICK ONE LINE FOR ASSIGMENT FOR DICTIONARY OF POSITIONS 
all_stations_name_red=all_stations_name[1:]
for station in all_stations_name_red:
# for station in ['Alewife']:

    print(station)
    depart_stat_lat=round(all_stations[all_stations['stop_name']==station]['stop_lat'].iloc[0],6)
    depart_stat_lon=round(all_stations[all_stations['stop_name']==station]['stop_lon'].iloc[0],6)
    print(depart_stat_lat,depart_stat_lon)

    station_weather=GetWeatherData(date_init,date_fina,depart_stat_lat,depart_stat_lon)
    station_weather=np.array(station_weather)
    
    with open('WEATHER_DATA/'+station+'_'+str(depart_stat_lat)+'_'+str(depart_stat_lon)+'.csv', mode='w') as weather_file:
        weather_file = csv.writer(weather_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # HEADER
        weather_file.writerow(["U_DATETIME","TEMP","PRECIP_INT","PRECIP_PRO","PRECIP_ACC","PRECIP_TYP"])

        for line in range(len(station_weather)):
            weather_file.writerow([station_weather[line,0],station_weather[line,1],station_weather[line,2],
                                   station_weather[line,3],station_weather[line,4],station_weather[line,5]])


Beaconsfield
42.335809 -71.140841
# NUMBER OF DAYS = 1338
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensit

NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
GovernmentCenter
42.359705 -71.059215
# NUMBER OF DAYS = 1338
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precip

NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NewtonHighlands
42.321687 -71.206063
# NUMBER OF DAYS = 1338
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipI

NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO temperature GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
NO precipIntensity GIVEN!!
